# ACM Coding Challenge Submission Info
This project will clean up the data from the cars_raw.csv file and then filter this data to the 75th percentile of the highest rated cars according to consumers which are calculated using a Bayesian model that factors in not only the average rating but the number of reviews aswell. This filtered data will then be analyzed to determine the most common characteristics of these highly rated cars and visuaized with the use of pie charts and graphs.

In [190]:
#Import statements and basic setup
import numpy as np 
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

# Data Cleanup
First I used the Pandas library to grab the data from the cars_raw.csv file and store it in a data file

In [191]:
df = pd.read_csv('../input/carsforsale/cars_raw.csv')
df.head()

Using the info() method, I can get a list of all the data columns and their respective datatypes which will be helpful information later on

In [192]:
df.info()

# Price
The price column is represented as a string due to the value having to be formatted with a currency and indexed with commas. Additionally some of the prices arent filled in making this data hard to analyze and compare. So first I removed any rows that dont contain a price value then I reformatted the remaining values to only consist of numeric characters then convert this value to an integer for analysis later on.

In [193]:
df = df[df['Price'] != 'Not Priced']
df['Price'] = df.Price.str.replace('$','') 
df['Price'] = df.Price.str.replace(',','').astype(int) 

# Used/New
In the data's Used/New column, each car is marked as either being used or certified under that cars manufacturer. This makes it hard to query whether a car is simply Used or New because there are many differnent car manufactureres given in the list. So I replaced the data of all new cars to simply say Certified for easier comparison.

In [194]:
df['Used/New'] = df['Used/New'].apply(lambda x: 'Certified' if 'Certified' in x else x)

# Drivetrain
For the drivetrain of each car, the seller may have inputted a different value when referring the the same thing (i.e front wheel drive, FWD, or Front-Wheel Drive). This can lead to inconsistencies in the analysis so I standerized all these drivetrains into either FWD, AWD, or RWD to make it more accurate for the program to analyze.

In [195]:
df = df[df['Drivetrain'] != '–']
def drive(x):
    if x == 'Front-wheel Drive' or x == 'Front Wheel Drive' or x == 'FWD':
        return 'FWD'
    if x == 'Four-wheel Drive' or x == 'All-wheel Drive' or x == '4WD' or x == 'AWD':
        return 'AWD'
    if x == 'Rear-wheel Drive' or x == 'RWD':
        return 'RWD'
df['Drivetrain'] = df['Drivetrain'].apply(drive)
df.Drivetrain

# Color
All the cars listed have a variety of colors that were all virtualy the same(i.e. "Shadow Black" or "Agate Black" just being Black), so to simplify analyzing this data, I got a list of all the unique colors present and generalized most of them to a basic color and classified everything else as other.

In [196]:
df['ExteriorColor'].unique()

In [197]:
def color(x):
    if 'red' in x.lower() or 'ruby' in x.lower() or 'lava' in x.lower():
        return 'Red'
    elif 'black' in x.lower() or 'shadow' in x.lower() or 'ebony' in x.lower():
        return 'Black'
    elif 'white' in x.lower() or 'blizzard' in x.lower():
        return 'White'
    elif 'blue' in x.lower() or 'aqua' in x.lower():
        return 'Blue'
    elif 'green' in x.lower() or 'moss' in x.lower() or 'olive' in x.lower():
        return 'Green'
    elif 'brown' in x.lower() or 'bronze' in x.lower() or 'walnut' in x.lower():
        return 'Brown'
    elif 'orange' in x.lower():
        return 'Orange'
    elif 'yellow' in x.lower():
        return 'Yellow'
    elif 'gray' in x.lower() or 'silver' in x.lower() or 'grey' in x.lower() or 'steel' in x.lower() or 'granite' in x.lower() or 'gun' in x.lower() or 'magnetic' in x.lower():
        return 'Gray'
    else:
        return 'Other'
df['ExteriorColor'] = df['ExteriorColor'].apply(color)

# Filtering the data
To get the 75th percentile of the highest user rated cars, we cannot just simply look at the customer rating because the amount of reviews also matters. Something with one 5 star review is not as highly rated has something with thirty 4.9 star reviews. To account for this I used a Bayesian model to calculate a weight to these ratings and sorted the data based off that weighted rating.

To get this weighted rating, I first needed to calculate the mean of all the ratings

In [198]:
mean = df['ConsumerRating'].mean()
mean

Then I needed to get a value to set the minimum number of reviews for a rating to be considered. To get this number I just got the average number of reviews each car had and set that to the minimum.

In [199]:
min = round(df['ConsumerReviews'].mean())
min

Then I used this simple formula for Bayesian rankings to calculate each cars weighted rating:

Weighted rating (WR) = (v / (v+m)) × R + (m / (v+m)) × C

Where:

R = Rating for the car

v = Number of reviews for the car

m = Minimum reviews required to be considered (133)

C = The mean rating across the data (4.7)

Then I also calulated the averages of all the categorical ratings within a car and added that to a new average ratings column. I then sorted the data by the weighted rating first then applied a secondary filter of the average rating to ensure the most accurate results.

In [200]:
df['WeightedConsumerReview'] = df.ConsumerReviews/(df.ConsumerReviews + min) * df.ConsumerRating +(min/(df.ConsumerReviews + min))*mean
df['AverageRatings'] = (df.ComfortRating + df.InteriorDesignRating + df.PerformanceRating + df.ValueForMoneyRating + df.ExteriorStylingRating + df.ReliabilityRating)/6
df = df.sort_values(['WeightedConsumerReview','AverageRatings'],ascending=False).drop(columns=['WeightedConsumerReview','AverageRatings'])

After the list was sorted, I filtered the list down to the 75th percentile of the highest weighted ratings and stored this new data set in another variable that will be used for the analysis.

In [201]:
x = int(round(len(df.index)*.25))
df_filtered = df.head(x)
df_filtered

# Analysis 
To format the graphs properly and avoid clutter, I constructed 2 functions to handle the percentage and labels. These functions only display the percent and label if the percent of the data was greater than 1.

In [202]:
def autopct(pct): 
    return ('%.2f' % pct + "%") if pct > 1 else ''

def labels(data):
   list = []
   for i in range(len(data)):
       if (data[i]*100/np.sum(data)) > 1 :
           list.append(data.index[i])
       else:
           list.append('')
   return list

# Make Analysis
I plotted the filtered data into a pie chart by the make of the cars. Here we can see that Honda is the most popular brand making up 21.09% of the data which is a very signifigant chunk with the next highest brand being Toyota at 13.66%. This shows an interesting pattern that will be expanded on in where the more lower to middle end vehicles are more abundant within the 75th percentile of the data.  

In [203]:
MakeValueCounts = df_filtered['Make'].value_counts()
MakeValueCounts.plot.pie(title='Make',ylabel="", figsize=(20, 20),autopct=autopct,labels=labels(MakeValueCounts),textprops={'fontsize': 14})

# MPG Analysis
To properly graph the MPG of the cars, I had to calculate each cars average MPG based off their minimum and maximum MPG by using the standard mean formula. Since this would result in a variety of numbers, I decided to group these values within ranges, but to get proper ranges that would display properly, I would have to see the shape of my data. So I plotted the averages in a bar graph to get a proper view of the distribution in order to set up the ranges

In [204]:
df_filtered['AvgMPG'] = (df['MinMPG'] + df['MaxMPG'])/2
df_filtered['AvgMPG'].value_counts().sort_index().plot(xlabel='AVG MPG', kind = 'bar',figsize=(20, 5))

Looking at the data, Its clear to see there is a cluster of data within the 20-30 MPG range, so I decided to make the ranges intervals of 5 starting from 0 all the way to 40. So then after I grouped all the data into their respective Average MPG range, I plotted the data into a piechartand found that a staggering 46.24% of the cars fell within the 20-24 MPG range. With the next highest being 23.44% in the 25-29 MPG range 

In [205]:
def GetMPGRange(AvgMPG):
    if(AvgMPG < 5):
        return '0 - 4'
    elif(AvgMPG < 10):
        return '5 - 9'
    elif(AvgMPG < 15):
        return '10 - 14'
    elif(AvgMPG < 20):
        return '15 - 19'
    elif(AvgMPG < 25):
        return '20 - 24'
    elif(AvgMPG < 30):
        return '25 - 29'
    elif(AvgMPG < 35):
        return '30 - 34'
    elif(AvgMPG < 40):
        return '35 - 39'
    else:
        return '40+'

df_filtered['AvgMPGRange'] = df_filtered['AvgMPG'].apply(GetMPGRange)
AvgMPGValueCounts = df_filtered['AvgMPGRange'].value_counts()
AvgMPGValueCounts.plot.pie(title='Average MPG',ylabel="", figsize=(15, 15),autopct=autopct,labels=labels(AvgMPGValueCounts),textprops={'fontsize': 14})
df_filtered=df_filtered.drop(columns=['AvgMPGRange','AvgMPG'])

# Price Analysis
I applied a similar methodology from the MPG column to the prices column where I grouped the data into ranges. I used intervals of 5,000 starting from 20,000 and going all the way up to 55,000. Then I plotted another pie chart based off these ranges and found that nearly 26.13% of the data fell within the 30,000-34,999 range and 22.59% fell within the 25,000-29,999 range. This accounts for 48.72% of the data which falls within the range of 25,000-34,999 while the more expensive cars being 45,000+ only accounted for only 12.22%. This continues the trend that more high end and expensive cars are actually less highly rated compared to cars in the low-middle range.

In [206]:
df_filtered['Price'].max()
def GetPriceRange(Price):
    if(Price < 20000):
        return '\$0 - \$19,999'
    elif(Price < 25000):
        return '\$20,000 - \$24,999'
    elif(Price < 30000):
        return '\$25,000 - \$29,999'
    elif(Price < 35000):
        return '\$30,000 - \$34,999'
    elif(Price < 40000):
        return '\$35,000 - \$39,999'
    elif(Price < 45000):
        return '\$40,000 - \$44,999'
    elif(Price < 50000):
        return '\$45,000 - \$49,999'
    elif(Price < 55000):
        return '\$50,000 - \$54,999'
    else:
        return '\$55,000+'
df_filtered['PriceRange'] = df_filtered['Price'].apply(GetPriceRange)
PriceValueCounts = df_filtered['PriceRange'].value_counts()
PriceValueCounts.plot.pie(title='Price',ylabel="", figsize=(10, 10),autopct=autopct,labels=labels(PriceValueCounts),textprops={'fontsize': 14})
df_filtered=df_filtered.drop(columns=['PriceRange'])

# Color Analysis
For colors I plotted the data that was cleaned up in a horizontal bargraph to visually show which colors were the most present and found that nuetral colors such as black, gray, and white were the overwhelming majority of the data.

In [207]:
colors =['Gray', 'Black', 'White', 'Blue', 'Red', 'Pink', 'Green', 'Brown', 'Orange', 'Yellow']
ColorValueCounts = df_filtered['ExteriorColor'].value_counts()
ColorValueCounts.sort_values().plot(title='Exterior Color',ylabel="Count",xlabel='Color', kind = 'barh',figsize=(7, 7), color=colors[::-1], edgecolor = "black", fontsize=14)

I then plotted the data into a pie chart so get the percentages of the colors and found that gray, black, and white made up 30.36%, 29.5%, and 16.48% respectively. 

In [208]:
ColorValueCounts.plot.pie(title='Exterior Color',ylabel="", figsize=(10, 10),autopct=autopct,labels=labels(ColorValueCounts),textprops={'fontsize': 14})

# Drivetrain Analysis
Here I plotted the drivetrain data into a piechart and found that a majority of 61.7% of the cars were all wheel drive. 

In [209]:
DrivetrainValueCounts = df_filtered['Drivetrain'].value_counts()
DrivetrainValueCounts.plot.pie(title='Drivetrain',ylabel="", figsize=(10, 10),autopct=autopct,labels=labels(DrivetrainValueCounts),textprops={'fontsize': 14})

# Condition Analysis

Here I plotted the condition of the car and found that a staggering majority of the cars were used, furthering the point that a higher quality newer cars are less present in the 75th percentile of highest rated cars

In [210]:
ConditionValueCounts = df_filtered['Used/New'].value_counts()
ConditionValueCounts.plot(title='Condition',ylabel="Count",xlabel='Color', kind = 'bar',figsize=(7, 7), edgecolor = "black", fontsize=14)

# Conclusion
In conclusion, after analyzing the data of the 75th percentile of the highest rated cars from within the data set of cars_raw.csv, Ive found that it largely consist of non luxury used cars that are of nuetral color and within a low to medium price range. This shows a trend of how higher end cars dont necessarily correlate to a high customer rating. However this analysis cannot be entirely accurate as the data source may have some factors that cause the data to be skewed.   